In [1]:
import requests
import pandas as pd
import numpy as np
import requests 
import os
from io import StringIO  # Import StringIO from the io module
import time  # Import the time module
from urllib.parse import quote  # To URL encode SMILES
import plotly.express as px
import plotly.io as pio
import matplotlib.colors as mcolors

Befor runing this code: 
1) create a .csv file containing a 'species_header' like 'species', 'names', etc. this column should contain the name of the species (or genuses by default) resolved by Open Tree of Life.
2) creta an empty folder (input_folder) and put the .csv file inside! 

That's all

DEFINE THE FOLLOWING PARAMETERS

In [1]:
#Paths
# Input folder containing CSV files
input_folder = 'C:/Users/quirosgu/Desktop/test/' #/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/V1V2/'

species_header = 'organism_species' #'query_otol_species' #put the column of your species
genus_header = 'query_otol_genus'
family_header = 'query_otol_family'

LOTUSDB = 'C:/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv' #'/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv'
LOTUSDB_rc = 'C:/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_rc.csv' #'/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/Yggdrasil/data_loc/LotusDB_inhouse_rc.csv'

# Output folder for the processed CSV files
output_folder = f'{input_folder}output_data/'

FIRST RECOVER THE Q CODES FROM WIKIDATA AND UPDATE THE METADATA FILE

In [7]:
# Function to process a single CSV file and retrieve Q codes
def process_csv_file(input_file, output_folder):
    # Read the original CSV file containing the species names
    df_species = pd.read_csv(input_file)
    
    # Define the Wikidata Query Service endpoint URL
    endpoint_url = "https://query.wikidata.org/sparql"
    
    # Initialize an empty list to store the Q codes
    q_codes = []
    
    # Initialize a placeholder value for cases where no Q code is found
    placeholder_value = "Not Found"
    
    # Define the number of requests allowed per minute (adjust according to Wikidata's rate limits)
    requests_per_minute = 30
    
    # Iterate through the species names in the CSV file
    for index, row in df_species.iterrows():
        species_name = row[species_header]
    
        # Check if the species_name is a valid string
        if isinstance(species_name, str):
            # Query Wikidata to retrieve the corresponding Q code for the species
            sparql_query = f"""
            SELECT ?species ?speciesLabel WHERE {{
                
                ?species rdfs:label "{species_name}"@en.
                SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
            }}
            """
    
            response = requests.get(endpoint_url, headers={'Accept': 'text/csv'}, params={'query': sparql_query})
    
            if response.status_code == 200:
                csv_data = response.text
                df_qcode = pd.read_csv(StringIO(csv_data))
    
                if not df_qcode.empty:
                    q_code = df_qcode['species'].values[0]  # Extract the Q code
                    q_codes.append(q_code)  # Append the Q code to the list
                    print(f"Retrieved Q code {q_code} for species: {species_name}")
                else:
                    q_codes.append(placeholder_value)  # Add a placeholder value
                    print(f"No Q code found for species: {species_name}")
            else:
                q_codes.append(placeholder_value)  # Add a placeholder value for failed requests
                print(f"Failed to retrieve Q code for species: {species_name}. Status code: {response.status_code}")
    
            # Implement rate limiting by waiting between requests
            time.sleep(60 / requests_per_minute)  # Wait for one minute divided by the allowed requests per minute
        else:
            q_codes.append(placeholder_value)  # Add a placeholder value for invalid species names
    
    # Add the Q codes hyperlinks to the original CSV with a new column
    df_species['wikidata_Qcode_hyperlink'] = q_codes
    
    # Add the Q codes only to the original CSV with a new column
    df_species['wikidata_Qcode_species'] = df_species['wikidata_Qcode_hyperlink'].str.rsplit('/', n=1).str[-1]
    
    # Save the updated CSV file with Q codes in the output folder
    output_csv_filename = os.path.join(output_folder, os.path.basename(input_file))
    df_species.to_csv(output_csv_filename, index=False)
    print(f"Saved updated CSV with Q codes to {output_csv_filename}")

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# List all CSV files in the input folder and process each one
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        input_file = os.path.join(input_folder, filename)
        process_csv_file(input_file, output_folder)

#once finished... check the lines for 'NOT FOUND' manually

Retrieved Q code http://www.wikidata.org/entity/Q15287263 for species: Acacia hayesii
Retrieved Q code http://www.wikidata.org/entity/Q8186515 for species: Acalypha aristata
Retrieved Q code http://www.wikidata.org/entity/Q1882542 for species: Acrocomia aculeata
Retrieved Q code http://www.wikidata.org/entity/Q4676527 for species: Acrostichum aureum
Retrieved Q code http://www.wikidata.org/entity/Q5657528 for species: Adelia triloba
Retrieved Q code http://www.wikidata.org/entity/Q11079071 for species: Aeschynomene americana
Retrieved Q code http://www.wikidata.org/entity/Q104912462 for species: Ageratina ixiocladon
Retrieved Q code http://www.wikidata.org/entity/Q4692127 for species: Ageratina ligustrina
Retrieved Q code http://www.wikidata.org/entity/Q1949712 for species: Ageratum conyzoides
Retrieved Q code http://www.wikidata.org/entity/Q159717 for species: Agrostis capillaris
Retrieved Q code http://www.wikidata.org/entity/Q15492162 for species: Albizia adinocephala
Retrieved Q co

KeyboardInterrupt: 

NOW, LETS USE THE QCODES TO RECOVER ALL THE COMPOUNDS (WITH REFERENCES) PRESENT IN EACH QCODE AND SAVE THEM AS INDIVIDUAL CSV FILES FROM LOTUS_DB FROZEN METADATA

In [10]:
# Load the LOTUSDB CSV
lotusdb_df = pd.read_csv(LOTUSDB, low_memory=False)

# Iterate through all CSV files in the input folder
for filename in os.listdir(output_folder):
    if filename.endswith('.csv'):
        input_file = os.path.join(output_folder, filename)

        # Load the CSV file with Q codes
        df_species = pd.read_csv(input_file)

        # Iterate through the Q codes in the species CSV
        for q_code in df_species['wikidata_Qcode_species']:  # Ensure this column name matches your species CSV
            # Skip processing if Q code is 'Not Found'
            if q_code == 'Not Found':
                continue

            # Filter LOTUSDB data for the current Q code
            filtered_lotusdb = lotusdb_df[lotusdb_df['wikidata_Qcode'] == q_code]  # Ensure this column name matches your LOTUSDB CSV

            # Group and aggregate the data
            grouped_df = filtered_lotusdb.groupby("structure_inchikey").agg({
                # Add all the aggregation rules here
                # Example:
                "structure_wikidata": "first",
                "structure_inchi": "first",
                "structure_smiles": "first",
                "structure_molecular_formula": "first",
                "structure_exact_mass": "first",
                "structure_xlogp": "first",
                "structure_smiles_2D": "first",
                "structure_cid": "first",
                "structure_nameIupac": "first",
                "structure_nameTraditional": "first",
                "structure_taxonomy_npclassifier_01pathway": "first",
                "structure_taxonomy_npclassifier_02superclass": "first",
                "structure_taxonomy_npclassifier_03class": "first",
                "organism_wikidata": "first",
                "organism_taxonomy_gbifid": "first",
                "organism_taxonomy_ncbiid": "first",
                "organism_taxonomy_ottid": "first",
                "organism_taxonomy_01domain": "first",
                "organism_taxonomy_02kingdom": "first",
                "organism_taxonomy_03phylum": "first",
                "organism_taxonomy_04class": "first",
                "organism_taxonomy_05order": "first",
                "organism_taxonomy_06family": "first",
                "organism_taxonomy_07tribe": "first",
                "organism_taxonomy_08genus": "first",
                "organism_taxonomy_09species": "first",
                "organism_taxonomy_10varietas": "first",
                "reference_wikidata": lambda x: "|".join(map(str, x)),
                "reference_doi": lambda x: "|".join(map(str, x))
            }).reset_index()
            
            #create a 'short_inchikey' column
            grouped_df['short_inchikey'] =  grouped_df['structure_inchikey'].str[:14]
            #create a 'chemical_superclass' column
            grouped_df['chemical_superclass'] =  grouped_df['structure_taxonomy_npclassifier_01pathway'] + '-' +  grouped_df['structure_taxonomy_npclassifier_02superclass']
            grouped_df['chemical_class'] =  grouped_df['structure_taxonomy_npclassifier_01pathway'] + '-' +  grouped_df['structure_taxonomy_npclassifier_03class']
            
            
            # Save the grouped data as a TSV file with the Q code as the filename
            output_subfolder = os.path.join(output_folder, 'species_data')
            os.makedirs(output_subfolder, exist_ok=True)
            output_filename = os.path.join(output_subfolder, f"{q_code}.tsv")
            grouped_df.to_csv(output_filename, index=False, sep='\t')

            print(f"Saved grouped data for Q code {q_code} to {output_filename}")


Saved grouped data for Q code Q15287263 to C:/Users/quirosgu/Desktop/Inmuno/output_data/species_data\Q15287263.tsv
Saved grouped data for Q code Q8186515 to C:/Users/quirosgu/Desktop/Inmuno/output_data/species_data\Q8186515.tsv
Saved grouped data for Q code Q1882542 to C:/Users/quirosgu/Desktop/Inmuno/output_data/species_data\Q1882542.tsv
Saved grouped data for Q code Q4676527 to C:/Users/quirosgu/Desktop/Inmuno/output_data/species_data\Q4676527.tsv
Saved grouped data for Q code Q5657528 to C:/Users/quirosgu/Desktop/Inmuno/output_data/species_data\Q5657528.tsv
Saved grouped data for Q code Q11079071 to C:/Users/quirosgu/Desktop/Inmuno/output_data/species_data\Q11079071.tsv
Saved grouped data for Q code Q104912462 to C:/Users/quirosgu/Desktop/Inmuno/output_data/species_data\Q104912462.tsv
Saved grouped data for Q code Q4692127 to C:/Users/quirosgu/Desktop/Inmuno/output_data/species_data\Q4692127.tsv
Saved grouped data for Q code Q1949712 to C:/Users/quirosgu/Desktop/Inmuno/output_data/s

In [3]:
# Create a subfolder for genus data
genus_subfolder = os.path.join(output_folder, 'genus_data')
os.makedirs(genus_subfolder, exist_ok=True)

# Iterate through all CSV files in the input folder
for filename in os.listdir(output_folder):
    if filename.endswith('.csv'):
        input_file = os.path.join(output_folder, filename)

        # Load the CSV file with Q codes
        df_species = pd.read_csv(input_file)

        # Iterate through the Q codes in the species CSV
        for genus in df_species[genus_header]:  # Ensure this column name matches your species CSV
            # Skip processing if Q code is 'Not Found'
            if genus == 'Not Found':
                continue

            # Filter LOTUSDB data for the current Q code
            filtered_lotusdb = lotusdb_df[lotusdb_df['organism_taxonomy_08genus'] == genus]  # Ensure this column name matches your LOTUSDB CSV

            # Group and aggregate the data at the genus level based on 'genus_header' column
            grouped_df = filtered_lotusdb.groupby("structure_inchikey").agg({
                # Add all the aggregation rules here
                # Example:
                "structure_wikidata": "first",
                "structure_inchi": "first",
                "structure_smiles": "first",
                "structure_molecular_formula": "first",
                "structure_exact_mass": "first",
                "structure_xlogp": "first",
                "structure_smiles_2D": "first",
                "structure_cid": "first",
                "structure_nameIupac": "first",
                "structure_nameTraditional": "first",
                "structure_taxonomy_npclassifier_01pathway": "first",
                "structure_taxonomy_npclassifier_02superclass": "first",
                "structure_taxonomy_npclassifier_03class": "first",
                "organism_wikidata": "first",
                "organism_taxonomy_gbifid": "first",
                "organism_taxonomy_ncbiid": "first",
                "organism_taxonomy_ottid": "first",
                "organism_taxonomy_01domain": "first",
                "organism_taxonomy_02kingdom": "first",
                "organism_taxonomy_03phylum": "first",
                "organism_taxonomy_04class": "first",
                "organism_taxonomy_05order": "first",
                "organism_taxonomy_06family": "first",
                "organism_taxonomy_07tribe": "first",
                "organism_taxonomy_08genus": "first",
                "organism_taxonomy_09species": "first",
                "organism_taxonomy_10varietas": "first",
                "reference_wikidata": lambda x: "|".join(map(str, x)),
                "reference_doi": lambda x: "|".join(map(str, x))
            }).reset_index()
            
            # Create a 'chemical_superclass' column
            grouped_df['chemical_superclass'] =  grouped_df['structure_taxonomy_npclassifier_01pathway'] + '-' +  grouped_df['structure_taxonomy_npclassifier_02superclass']
            grouped_df['chemical_class'] =  grouped_df['structure_taxonomy_npclassifier_01pathway'] + '-' +  grouped_df['structure_taxonomy_npclassifier_03class']
            
            # Save the grouped data as a TSV file in the genus_data subfolder
            output_filename = os.path.join(genus_subfolder, f"{genus}.tsv")
            grouped_df.to_csv(output_filename, index=False, sep='\t')

            print(f"Saved grouped data for genus {genus} to {output_filename}")


NameError: name 'lotusdb_df' is not defined

RECOVER THE FRECUENCE BY CHEMICAL CLASS AND ADDED TO THE GENERAL TABLE

In [3]:
# Step 1: Initialize variables to store data
total_compounds = {}
chemical_classes = {}
chemical_superclasses = {}
references = {}
hyperlinks = {}

# Step 2: Iterate through .tsv files in the folder

species_data_folder = os.path.join(output_folder, 'species_data')
for filename in os.listdir(species_data_folder):
    if filename.endswith(".tsv"):
        qcode = filename.split(".")[0]  # Extract Qcode from the filename

        # Load the .tsv file into a DataFrame
        df_compounds = pd.read_csv(os.path.join(species_data_folder, filename), sep='\t')

        # Step 3: Calculate frequencies of chemical classes for each Qcode (excluding 'not classified')
        filtered_classes = df_compounds[df_compounds['structure_taxonomy_npclassifier_03class'] != 'Not Classified']
        grouped = filtered_classes['structure_taxonomy_npclassifier_03class'].value_counts().reset_index()
        frequencies = grouped.apply(lambda row: f"{row.name} {row['structure_taxonomy_npclassifier_03class']}", axis=1)
        chemical_classes[qcode] = "|".join(frequencies)

        # Step 4: Calculate frequencies of chemical superclasses for each Qcode (excluding 'not classified')
        filtered_sclasses = df_compounds[df_compounds['structure_taxonomy_npclassifier_02superclass'] != 'Not Classified']
        sgrouped = filtered_sclasses['structure_taxonomy_npclassifier_02superclass'].value_counts().reset_index()
        frequencies = sgrouped.apply(lambda row: f"{row.name} {row['structure_taxonomy_npclassifier_02superclass']}", axis=1)
        chemical_superclasses[qcode] = "|".join(frequencies)
        chemical_classes[qcode] = "|".join(frequencies)
        
        # Step 5: Combine references and hyperlinks
        #references[qcode] = "|".join(df_compounds['reference_wikidata'].dropna())
        #hyperlinks[qcode] = "|".join(df_compounds['reference_doi'].dropna())

        # Step 6: Calculate the total number of reported compounds
        #total_compounds[qcode] = len(df_compounds)

# Step 7: Load the general info .csv into a DataFrame
csv_file = None
for filename in os.listdir(output_folder):
    if filename.endswith(".csv"):
        csv_file = os.path.join(output_folder, filename)
        break  # Stop searching after finding the first CSV file

if csv_file is not None:
    # Load the CSV file into the df_general_info DataFrame
    df_general_info = pd.read_csv(csv_file)

# Step 7: Create new columns for combined information
df_general_info['predicted_class'] = df_general_info['wikidata_Qcode'].map(chemical_classes)
df_general_info['predicted_superclass'] = df_general_info['wikidata_Qcode'].map(chemical_superclasses)
#df_general_info['references'] = df_general_info['wikidata_Qcode_species'].map(references)
#df_general_info['referenceLabel'] = df_general_info['wikidata_Qcode_species'].map(hyperlinks)

# Step 7: Save the updated DataFrame to the .csv file

# Replace values matching the pattern with an empty cell
df_general_info['predicted_class'] = df_general_info['predicted_class'].replace('index|predicted_class', '')
df_general_info['predicted_superclass'] = df_general_info['predicted_superclass'].replace('index|predicted_superclass', '')

#add reported compounds to each Q code
LotusDB_rc = pd.read_csv(LOTUSDB_rc)

spDB = LotusDB_rc[['wikidata_Qcode', 'Reported_comp_Species']].drop_duplicates()
gDB = LotusDB_rc[['organism_taxonomy_08genus', 'Reported_comp_Genus']].drop_duplicates()
fDB = LotusDB_rc[['organism_taxonomy_06family', 'Reported_comp_Family']].drop_duplicates()

df = pd.merge(df_general_info, spDB,
            how= 'left', left_on='wikidata_Qcode', right_on='wikidata_Qcode')#.drop_duplicates(subset=['CODE'])
df.drop('wikidata_Qcode', axis=1, inplace=True)

df = pd.merge(df, gDB,
            how= 'left', left_on=genus_header, right_on='organism_taxonomy_08genus')#.drop_duplicates(subset=['full_species'])
df.drop('organism_taxonomy_08genus', axis=1, inplace=True)

df = pd.merge(df, fDB,
            how= 'left', left_on=family_header, right_on='organism_taxonomy_06family')#.drop_duplicates(subset=['full_species'])
df.drop('organism_taxonomy_06family', axis=1, inplace=True)

output_csv_file = output_folder + 'Full_results.csv'
df.to_csv(output_csv_file, index=False, sep =',')

Generate une plot by species (samples) and the chemical class

In [2]:
# Define base colors for each Pathway

pathway_colors = {
    'Terpenoids': ('#618264', '#D0E7D2'),  # Green start and lighter green end
    'Alkaloids': ('#305F72', '#5CBCE2'),   # Blue start and lighter blue end  
    'Shikimates and Phenylpropanoids': ('#80558C', '#CBA0AE'),  # Purple start and lighter purple end
    'Polyketides': ('#EF4B4B', '#EC8F6A'),  # Red start and lighter purple end
    'Fatty acids': ('#FF6C22', '#FF9209'),  # Orange start and lighter purple end
    'Amino acids and Peptides': ('#F4E869', '#FAF2D3'),  # Yellow start and lighter purple end
    'Carbohydrates': ('#65451F','#C8AE7D')  # Brown start and lighter purple end
}

def interpolate_color(color1, color2, factor: float):
    """Interpolate between two colors"""
    color1 = np.array(mcolors.to_rgb(color1))
    color2 = np.array(mcolors.to_rgb(color2))
    return mcolors.to_hex((1 - factor) * color1 + factor * color2)

def generate_shades(pathway, num_shades):
    base_color, end_color = pathway_colors.get(pathway, ('gray', 'lightgray'))
    if num_shades == 1:
        return [base_color]  # Return the base color if only one shade is requested
    shades = []
    for i in range(num_shades):
        factor = i / (num_shades - 1)
        shades.append(interpolate_color(base_color, end_color, factor))
    return shades

def split_chemical_superclass(row):
    # Check if the value is a string before splitting
    if isinstance(row['chemical_superclass'], str):
        parts = row['chemical_superclass'].split('-')
        if len(parts) == 2:
            return parts[0], parts[1]  # Pathway and Superclass are present
        else:
            return parts[0], 'Unknown'  # Only Pathway is present, or the format is not as expected
    else:
        # Return default values if the entry is NaN or not a string
        return 'Unknown', 'Unknown'

In [8]:
# Step 1: Initialize an empty DataFrame to store all data
all_data = pd.DataFrame(columns=['organism_taxonomy_09species', 'organism_taxonomy_08genus','organism_taxonomy_06family', 'chemical_superclass'])

# Step 2: Read data from all .tsv files in the output_folder
species_data_folder = os.path.join(output_folder, 'species_data')
for filename in os.listdir(species_data_folder):
    if filename.endswith(".tsv"):
        file_data = pd.read_csv(os.path.join(species_data_folder, filename), sep='\t')
        all_data = pd.concat([all_data, file_data])

# Step 3: Rename the "queried_taxaLabel" column to "species"
all_data.rename(columns={'organism_taxonomy_09species': 'species', 'organism_taxonomy_08genus': 'genus','organism_taxonomy_06family': 'family',}, inplace=True)

# Apply the function to each row
all_data[['Pathway', 'superclass']] = all_data.apply(lambda row: split_chemical_superclass(row), axis=1, result_type='expand')

# Step 4: Process data for color mapping
color_map = {}
for pathway, superclasses in all_data.groupby('Pathway')['superclass'].unique().items():
    shades = generate_shades(pathway, len(superclasses))
    for superclass, shade in zip(superclasses, shades):
        color_map[f"{pathway}-{superclass}"] = shade

# Step 5: Group and aggregate data to calculate recurrence
agg_data = all_data.groupby(['species', 'genus', 'family', 'chemical_superclass']).size().reset_index(name='recurrence')

#unique_species = agg_data['species'].unique()


# Concatenate genus and family for the y-axis labels
#agg_data['genus_family'] = agg_data['family'] + ' > ' + agg_data['genus'] + ' > ' + agg_data['species']
agg_data['genus_family'] = agg_data['family'] + ' - ' + agg_data['species']

# Sort the agg_data DataFrame by 'species' and 'chemical_superclass'
agg_data_sorted = agg_data.sort_values(by=['chemical_superclass'])

# Step 6: Create the stacked barplot with the custom color palette
fig = px.bar(agg_data_sorted, y='genus_family', x='recurrence',
             #title='Stacked Barplot of Predicted Superclasses Occurrence for Species',
             labels={'recurrence': 'Recurrence'},
             color='chemical_superclass',
             color_discrete_map=color_map,
             orientation='h')

# Customize the order of species on the y-axis

#fig.update_yaxes(categoryorder='array', categoryarray=unique_species)
# Get unique values of 'genus_family_species' for ordering on the y-axis
unique_genus_family_species = sorted(agg_data_sorted['genus_family'].unique()) #agg_data_sorted['genus_family'].unique()

fig.update_yaxes(categoryorder='array', categoryarray=unique_genus_family_species)

# Set species labels in italics
#fig.update_layout(yaxis=dict(tickmode='array',
#                            tickvals=list(range(len(unique_genus_family_species))),
#                            ticktext=[f'<i>{species}</i>' for species in unique_genus_family_species],
#                            ticklabelposition='outside', 
                            #tickangle= 0,
                            #autorange="reversed",  # Reverse the y-axis to align labels to the left
                            #side='left'
#))


# Set species labels in italics and justify to the left
ticktext = []
for species in unique_genus_family_species:
    family, species_part = species.split(' - ')
    ticktext.append(f'{family} - <i>{species_part}</i>')  # Italicize only the species part

fig.update_layout(yaxis=dict(
    tickmode='array',
    tickvals=list(range(len(unique_genus_family_species))),
    ticktext=ticktext,
   # ticklabelposition='inside'  # Align tick labels to the left
))
# Set species labels in italics
#fig.update_layout(yaxis=dict(tickmode='array',
#                              tickvals=list(range(len(unique_species))),
#                              ticktext=[f'<i>{species}</i>' for species in unique_species]
#                              ))

# Modify the y-axis label
fig.update_yaxes(title_text='<i>Species<i>')

# Set a white background
fig.update_layout(plot_bgcolor='white')

# Modify the size of the figure
fig.update_layout(width=2000, height=2500)

# Define font style
font_style = dict(family="Times New Roman", size=20, color="black")

# Update the layout with the font style
fig.update_layout(
    title_font=font_style,  # Title font style
    font=font_style,  # Axis labels and tick labels font style
    yaxis=dict(tickfont=font_style, ticklen=50)  # Y-axis tick labels font style
)


# Save the figure as an HTML file
fig.write_html(f'{output_folder}Wikidata_superclass_barplot_species.html')

# Save the image in high resolution with specific dimensions
#fig.write_image('Wikidata_superclass_barplot_species.png', format= 'png', width=1000, height=1000, scale=3) 
# Show the figure
fig.show()

C:\Users\quirosgu\AppData\Local\Temp\ipykernel_13672\2488201323.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

C:\Users\quirosgu\AppData\Local\Temp\ipykernel_13672\2488201323.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

C:\Users\quirosgu\AppData\Local\Temp\ipykernel_13672\2488201323.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the resu

In [8]:

# Step 1: Initialize an empty DataFrame to store all data
all_data = pd.DataFrame(columns=['organism_taxonomy_09species', 'organism_taxonomy_08genus','organism_taxonomy_06family', 'chemical_superclass'])

# Step 2: Read data from all .tsv files in the output_folder
species_data_folder = os.path.join(output_folder, 'species_data')
for filename in os.listdir(species_data_folder):
    if filename.endswith(".tsv"):
        file_data = pd.read_csv(os.path.join(species_data_folder, filename), sep='\t')
        all_data = pd.concat([all_data, file_data])

# Step 3: Rename the "queried_taxaLabel" column to "species"
all_data.rename(columns={'organism_taxonomy_09species': 'species', 'organism_taxonomy_08genus': 'genus','organism_taxonomy_06family': 'family',}, inplace=True)

# Apply the function to each row
all_data[['Pathway', 'superclass']] = all_data.apply(lambda row: split_chemical_superclass(row), axis=1, result_type='expand')

# Step 4: Process data for color mapping
color_map = {}
for pathway, superclasses in all_data.groupby('Pathway')['superclass'].unique().items():
    shades = generate_shades(pathway, len(superclasses))
    for superclass, shade in zip(superclasses, shades):
        color_map[f"{pathway}-{superclass}"] = shade

# Step 5: Group and aggregate data to calculate recurrence
agg_data = all_data.groupby(['species', 'genus', 'family', 'chemical_superclass']).size().reset_index(name='recurrence')

#unique_species = agg_data['species'].unique()


# Concatenate genus and family for the y-axis labels
#agg_data['genus_family'] = agg_data['family'] + ' > ' + agg_data['genus'] + ' > ' + agg_data['species']
agg_data['genus_family'] = agg_data['family'] + ' - ' + agg_data['species']

# Sort the agg_data DataFrame by 'species' and 'chemical_superclass'
agg_data_sorted = agg_data.sort_values(by=['chemical_superclass'])
# Create the circular bar plot
fig = px.bar_polar(agg_data_sorted, r='recurrence', theta='genus_family',
                   color='chemical_superclass', template='plotly',
                   title='Circular Bar Plot of Predicted Superclasses Occurrence for Species',
                   color_discrete_map=color_map)

# Update the layout
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            showticklabels=True,
            tickmode='array',
            tickvals=[0, max(agg_data_sorted['recurrence'])],
            ticktext=['0', str(max(agg_data_sorted['recurrence']))]
        )
    )
)

# Modify the size of the figure
fig.update_layout(width=1500, height=1500)

# Show the figure
fig.show()


C:\Users\quirosgu\AppData\Local\Temp\ipykernel_10156\319300670.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

C:\Users\quirosgu\AppData\Local\Temp\ipykernel_10156\319300670.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

C:\Users\quirosgu\AppData\Local\Temp\ipykernel_10156\319300670.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result 

In [49]:

output_species = 'C:/Users/quirosgu/Desktop/Inmuno/output_data/species_data'

# Step 1: Read data from all .tsv files in the output_folder
all_data = pd.concat([pd.read_csv(os.path.join(output_species, filename), sep='\t') 
                        for filename in os.listdir(output_species) if filename.endswith(".tsv")])

# Step 2: Rename the "organism_taxonomy_09species" column to "species"
all_data.rename(columns={'organism_taxonomy_09species': 'species'}, inplace=True)

# Remove 'API Error-API Error' and 'Not Classified-Not Classified'
all_data = all_data[~all_data['chemical_superclass'].isin(['API Error-API Error', 'Not Classified-Not Classified'])]

# Apply the function to each row
all_data[['Pathway', 'superclass']] = all_data.apply(lambda row: split_chemical_superclass(row), axis=1, result_type='expand')

# Step 4: Process data for color mapping
color_map = {}
for pathway, superclasses in all_data.groupby('Pathway')['superclass'].unique().items():
    shades = generate_shades(pathway, len(superclasses))
    for superclass, shade in zip(superclasses, shades):
        color_map[f"{pathway}-{superclass}"] = shade

# Step 5: Group and aggregate data to calculate recurrence
agg_data = all_data.groupby(['species', 'chemical_superclass']).size().reset_index(name='recurrence')

# Normalize the recurrence values within each species group
agg_data['recurrence_normalized'] = agg_data.groupby('species')['recurrence'].transform(lambda x: x / x.sum()) * 100

# Convert 'species' column to categorical data
agg_data['species'] = pd.Categorical(agg_data['species'], categories=agg_data['species'].unique(), ordered=True)

# Get unique species names
unique_species = agg_data['species'].unique()

# Get unique chemical superclasses and sort them alphabetically
unique_superclasses = sorted(agg_data['chemical_superclass'].unique())

# Calculate total recurrence for each species
total_recurrence_per_species = agg_data.groupby('species')['recurrence'].sum()

# Create the stacked barplot with the custom color palette
fig = px.bar(agg_data, y='species', x='recurrence_normalized',
                title='Normalized Stacked Barplot of Predicted Superclasses Occurrence for Species',
                labels={'recurrence_normalized': 'Recurrence'},
                color='chemical_superclass',
                color_discrete_map=color_map,
                category_orders={'chemical_superclass': unique_superclasses},
                orientation='h')

# Modify the y-axis label
fig.update_yaxes(title_text='<i>Species<i>')

# Set species labels in italics
fig.update_layout(yaxis=dict(tickmode='array',
                                tickvals=list(range(len(unique_species))),
                                ticktext=[f'<i>{species}</i>' for species in unique_species]
                                ))

# Set a white background
fig.update_layout(plot_bgcolor='white')

# Get the maximum value of the x-axis
max_x = agg_data['recurrence_normalized'].max()

for species, total_recurrence in total_recurrence_per_species.items():
    # Find the maximum recurrence for this species
    max_recurrence = agg_data[agg_data['species'] == species]['recurrence_normalized'].max()

    # Position the annotation at the end of the bar with a slight offset
    x_position = max_x + 2
    if x_position > 100:  # Ensure the label is not outside the plot area
        x_position = 100  # Set it to 100 if it exceeds the maximum x-axis value

    # Add annotation to the plot
    fig.add_annotation(
        x=x_position,
        y=species,
        text=f'Total compounds: {total_recurrence}',
        showarrow=False,
        font=dict(size=10, color='black'),
        xanchor='left',
        yanchor='middle'
    )

# Modify the size of the figure
fig.update_layout(width=1500, height=1500)

# Save the figure as an HTML file
fig.write_html(f'{output_folder}Wikidata_superclass_barplot_species_normalized.html')

# Show the figure
fig.show()

# Step 1: Read data from all .tsv files in the output_folder
all_data = pd.concat([pd.read_csv(os.path.join(output_folder, filename), sep='\t') 
                        for filename in os.listdir(output_folder) if filename.endswith(".tsv")])

# Step 2: Rename the "organism_taxonomy_09species" column to "species"
all_data.rename(columns={'organism_taxonomy_09species': 'species'}, inplace=True)

# Remove 'API Error-API Error' and 'Not Classified-Not Classified'
all_data = all_data[~all_data['chemical_superclass'].isin(['API Error-API Error', 'Not Classified-Not Classified'])]

# Apply the function to each row
all_data[['Pathway', 'superclass']] = all_data.apply(lambda row: split_chemical_superclass(row), axis=1, result_type='expand')

# Step 4: Process data for color mapping
color_map = {}
for pathway, superclasses in all_data.groupby('Pathway')['superclass'].unique().items():
    shades = generate_shades(pathway, len(superclasses))
    for superclass, shade in zip(superclasses, shades):
        color_map[f"{pathway}-{superclass}"] = shade

# Step 5: Group and aggregate data to calculate recurrence
agg_data = all_data.groupby(['species', 'chemical_superclass']).size().reset_index(name='recurrence')

# Normalize the recurrence values within each species group
agg_data['recurrence_normalized'] = agg_data.groupby('species')['recurrence'].transform(lambda x: x / x.sum()) * 100

# Convert 'species' column to categorical data
agg_data['species'] = pd.Categorical(agg_data['species'], categories=agg_data['species'].unique(), ordered=True)

# Get unique species names
unique_species = agg_data['species'].unique()

# Get unique chemical superclasses and sort them alphabetically
unique_superclasses = sorted(agg_data['chemical_superclass'].unique())

# Calculate total recurrence for each species
total_recurrence_per_species = agg_data.groupby('species')['recurrence'].sum()

# Create the stacked barplot with the custom color palette
fig = px.bar(agg_data, y='species', x='recurrence_normalized',
                title='Normalized Stacked Barplot of Predicted Superclasses Occurrence for Species',
                labels={'recurrence_normalized': 'Recurrence'},
                color='chemical_superclass',
                color_discrete_map=color_map,
                category_orders={'species': unique_species, 'chemical_superclass': unique_superclasses},
                orientation='h')

# Modify the y-axis label
fig.update_yaxes(title_text='<i>Species<i>')

# Set species labels in italics
fig.update_layout(yaxis=dict(tickmode='array',
                                tickvals=list(range(len(unique_species))),
                                ticktext=[f'<i>{species}</i>' for species in unique_species]
                                ))

# Set a white background
fig.update_layout(plot_bgcolor='white')

# Get the maximum value of the x-axis
max_x = agg_data['recurrence_normalized'].max()

for species, total_recurrence in total_recurrence_per_species.items():
    # Find the maximum recurrence for this species
    max_recurrence = agg_data[agg_data['species'] == species]['recurrence_normalized'].max()

    # Position the annotation at the end of the bar with a slight offset
    x_position = max_x + 2
    if x_position > 100:  # Ensure the label is not outside the plot area
        x_position = 100  # Set it to 100 if it exceeds the maximum x-axis value

    # Add annotation to the plot
    fig.add_annotation(
        x=x_position,
        y=species,
        text=f'Total compounds: {total_recurrence}',
        showarrow=False,
        font=dict(size=10, color='black'),
        xanchor='left',
        yanchor='middle'
    )

# Modify the size of the figure
fig.update_layout(width=1500, height=1500)

# Save the figure as an HTML file
fig.write_html(f'{output_folder}Wikidata_superclass_barplot_species_normalized.html')

# Show the figure
fig.show()

C:\Users\quirosgu\AppData\Local\Temp\ipykernel_11824\3397825090.py:4: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

C:\Users\quirosgu\AppData\Local\Temp\ipykernel_11824\3397825090.py:39: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



ValueError: No objects to concatenate

In [ ]:
heatmap_pathway_species(output_folder):
    """
    Process data from .tsv files in the output folder and visualize it with a normalized stacked barplot.

    Parameters:
    output_folder (str): Path to the folder containing .tsv files.

    Returns:
    None
    """
    # Step 1: Initialize an empty DataFrame to store all data
    all_data = pd.DataFrame(columns=['organism_taxonomy_09species', 'structure_taxonomy_npclassifier_01pathway'])

    # Step 2: Read data from all .tsv files in the output_folder
    species_data_folder = os.path.join(output_folder)
    for filename in os.listdir(species_data_folder):
        if filename.endswith(".tsv"):
            file_data = pd.read_csv(os.path.join(species_data_folder, filename), sep='\t')
            all_data = pd.concat([all_data, file_data])

    # Step 3: Rename the "queried_taxaLabel" column to "species"
    all_data.rename(columns={'organism_taxonomy_09species': 'species'}, inplace=True)
    all_data.rename(columns={'structure_taxonomy_npclassifier_01pathway': 'Pathway'}, inplace=True)

    # Remove 'API Error-API Error' and 'Not Classified-Not Classified'
    all_data = all_data[(all_data['Pathway'] != 'API Error') & (all_data['Pathway'] != 'Not Classified')]

    # Step 4: Group and aggregate data to calculate recurrence
    agg_data = all_data.groupby(['species', 'Pathway']).size().reset_index(name='recurrence')

    # Normalize the recurrence values within each species group
    agg_data['recurrence_normalized'] = agg_data.groupby('species')['recurrence'].transform(lambda x: x / x.sum())*100

    # Convert 'species' column to categorical data
    agg_data['species'] = pd.Categorical(agg_data['species'], categories=agg_data['species'].unique(), ordered=True)

    
    # Create a DataFrame with all possible combinations of species and pathways
    all_species = all_data['species'].unique()
    all_pathways = all_data['Pathway'].unique()
    all_combinations = pd.DataFrame([(species, pathway) for species in all_species for pathway in all_pathways],
                                    columns=['species', 'Pathway'])

    # Merge the all_combinations DataFrame with the aggregated data to include all species
    merged_data = pd.merge(all_combinations, agg_data, on=['species', 'Pathway'], how='left')

    # Fill missing values (NaN) with 0 for recurrence and recurrence_normalized columns
    merged_data['recurrence'].fillna(0, inplace=True)
    merged_data['recurrence_normalized'].fillna(0, inplace=True)

    # Pivot the merged data to have 'species' as rows and 'Pathway' as columns
    pivot_data = merged_data.pivot_table(index='species', columns='Pathway', values='recurrence_normalized', fill_value=0)

    # Create the heatmap
    fig = px.imshow(pivot_data,
                    labels=dict(x="Pathway", y="Species", color="Normalized Recurrence"),
                    x=pivot_data.columns,
                    y=pivot_data.index,
                    color_continuous_scale='tealrose',  # You can choose any other color scale
                    title='Heatmap of Normalized Recurrence of Pathways for Species')

    # Modify the y-axis label
    fig.update_yaxes(title_text='<i>Species<i>')

    # Set species labels in italics
    fig.update_layout(yaxis=dict(tickmode='array', 
                                tickvals=list(range(len(unique_species))), 
                                ticktext=[f'<i>{species}</i>' for species in unique_species]
                                ))
    # Modify the size of the figure
    fig.update_layout(width=1500, height=1500)

    # Save the figure as an HTML file
    fig.write_html(f'{output_folder}Wikidata_pathway_heatmap_species_normalized.html')

    # Show the figure
    fig.show()

Generate une plot by species (samples) and the Pathway

In [33]:
# Define custom colors for the 7 pathway categories
pathway_colors = {
    'Terpenoids': '#618264',  # Green start and lighter green end
    'Alkaloids': '#305F72',   # Blue start and lighter blue end
    'Shikimates and Phenylpropanoids': '#80558C',  # Purple start and lighter purple end
    'Polyketides': '#EF4B4B',  # Red start and lighter purple end
    'Fatty acids': '#FF6C22',  # Orange start and lighter purple end
    'Amino acids and Peptides': '#F4E869',  # Yellow start and lighter purple end
    'Carbohydrates': '#65451F' # Brown start and lighter purple end
}

# Step 1: Initialize an empty DataFrame to store all data
all_data = pd.DataFrame(columns=['organism_taxonomy_09species', 'structure_taxonomy_npclassifier_01pathway'])

# Step 2: Read data from all .tsv files in the output_folder
species_data_folder = os.path.join(output_folder, 'species_data')
for filename in os.listdir(species_data_folder):
    if filename.endswith(".tsv"):
        file_data = pd.read_csv(os.path.join(species_data_folder, filename), sep='\t')
        all_data = pd.concat([all_data, file_data])

# Step 3: Rename the "queried_taxaLabel" column to "species"
all_data.rename(columns={'organism_taxonomy_09species': 'species'}, inplace=True)
all_data.rename(columns={'structure_taxonomy_npclassifier_01pathway': 'Pathway'}, inplace=True)
# Remove 'API Error-API Error' and 'Not Classified-Not Classified'
all_data = all_data[(all_data['Pathway'] != 'API Error') & (all_data['Pathway'] != 'Not Classified')]

# Step 4: Group and aggregate data to calculate recurrence
agg_data = all_data.groupby(['species', 'Pathway']).size().reset_index(name='recurrence')


# Get unique species names
unique_species = agg_data['species'].unique()

# Step 5: Create the stacked barplot with custom colors
fig = px.bar(agg_data, y=agg_data['species'].apply(lambda x: f"<i>{x}</i>"), x='recurrence',
             title='Stacked Barplot of Predicted Pathways Ocurrence for Species',
             labels={'recurrence': 'Recurrence'},
             color='Pathway',
             color_discrete_map=pathway_colors,  # Use custom colors
             category_orders={'Pathway': sorted(agg_data['Pathway'].unique())},
             orientation='h'
            )

# Set species labels in italics
fig.update_layout(yaxis=dict(tickmode='array',
                                  tickvals=list(range(len(unique_species))),
                                  ticktext=[f'<i>{species}</i>' for species in unique_species]
                                  ))                            

# Modify the y-axis label
fig.update_yaxes(title_text='<i>Species<i>')

# Set a white background
fig.update_layout(plot_bgcolor='white')

# Modify the size of the figure
fig.update_layout(width=1500, height=1500)

# Define font style
font_style = dict(family="Times New Roman", size=18, color="black")

# Set the font style for title
fig.update_layout(title_font=font_style)

# Save the figure as an HTML file
fig.write_html(f'{output_folder}Wikidata_pathway_barplot.html')

# Show or save the figure
fig.show()

C:\Users\quirosgu\AppData\Local\Temp\ipykernel_11824\522357952.py:20: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

C:\Users\quirosgu\AppData\Local\Temp\ipykernel_11824\522357952.py:20: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

C:\Users\quirosgu\AppData\Local\Temp\ipykernel_11824\522357952.py:20: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the resu

In [ ]:
barplot_pathway_species_normalized(output_folder):
    """
    Process data from .tsv files in the output folder and visualize it with a normalized stacked barplot.

    Parameters:
    output_folder (str): Path to the folder containing .tsv files.

    Returns:
    None
    """
    # Define custom colors for the 7 pathway categories
    pathway_colors = {
        'Terpenoids': '#618264',  # Green start and lighter green end
        'Alkaloids': '#305F72',   # Blue start and lighter blue end
        'Shikimates and Phenylpropanoids': '#80558C',  # Purple start and lighter purple end
        'Polyketides': '#EF4B4B',  # Red start and lighter purple end
        'Fatty acids': '#FF6C22',  # Orange start and lighter purple end
        'Amino acids and Peptides': '#F4E869',  # Yellow start and lighter purple end
        'Carbohydrates': '#65451F' # Brown start and lighter purple end
    } 
    # Step 1: Initialize an empty DataFrame to store all data
    all_data = pd.DataFrame(columns=['organism_taxonomy_09species', 'structure_taxonomy_npclassifier_01pathway'])

    # Step 2: Read data from all .tsv files in the output_folder
    species_data_folder = os.path.join(output_folder)
    for filename in os.listdir(species_data_folder):
        if filename.endswith(".tsv"):
            file_data = pd.read_csv(os.path.join(species_data_folder, filename), sep='\t')
            all_data = pd.concat([all_data, file_data])

    # Step 3: Rename the "queried_taxaLabel" column to "species"
    all_data.rename(columns={'organism_taxonomy_09species': 'species'}, inplace=True)
    all_data.rename(columns={'structure_taxonomy_npclassifier_01pathway': 'Pathway'}, inplace=True)

    # Remove 'API Error-API Error' and 'Not Classified-Not Classified'
    all_data = all_data[(all_data['Pathway'] != 'API Error') & (all_data['Pathway'] != 'Not Classified')]

    # Step 4: Group and aggregate data to calculate recurrence
    agg_data = all_data.groupby(['species', 'Pathway']).size().reset_index(name='recurrence')

    # Normalize the recurrence values within each species group
    agg_data['recurrence_normalized'] = agg_data.groupby('species')['recurrence'].transform(lambda x: x / x.sum())*100

    # Convert 'species' column to categorical data
    agg_data['species'] = pd.Categorical(agg_data['species'], categories=agg_data['species'].unique(), ordered=True)

    # Get unique species names
    unique_species = agg_data['species'].unique()

    # Get unique pathways and sort them alphabetically
    unique_pathways = sorted(agg_data['Pathway'].unique())

    # Calculate total recurrence for each species
    total_recurrence_per_species = agg_data.groupby('species')['recurrence'].sum()


    # Step 5: Create the stacked barplot with custom colors
    fig = px.bar(agg_data, y='species', x='recurrence_normalized',
                title='Normalized Stacked Barplot of Predicted Pathways Ocurrence for Species',
                labels={'recurrence_normalized': 'Recurrence'},
                color='Pathway',
                color_discrete_map=pathway_colors,  # Use custom colors
                category_orders={'species': unique_species, 'Pathway' :unique_pathways },
                orientation='h'
                )

    # Modify the y-axis label
    fig.update_yaxes(title_text='<i>Species<i>')

    # Set species labels in italics
    fig.update_layout(yaxis=dict(tickmode='array', 
                                tickvals=list(range(len(unique_species))), 
                                ticktext=[f'<i>{species}</i>' for species in unique_species]
                                ))
    # Get the maximum value of the x-axis
    max_x = agg_data['recurrence_normalized'].max()

    for species, total_recurrence in total_recurrence_per_species.items():
        # Find the maximum recurrence for this species
        max_recurrence = agg_data[agg_data['species'] == species]['recurrence_normalized'].max()
        
        # Position the annotation at the end of the bar with a slight offset
        x_position = max_x + 2
        if x_position > 100:  # Ensure the label is not outside the plot area
            x_position = 100  # Set it to 100 if it exceeds the maximum x-axis value

        # Add annotation to the plot
        fig.add_annotation(
            x=x_position,
            y=species,
            text=f'Total compounds: {total_recurrence}',
            showarrow=False,
            font=dict(size=10, color='black'),
            xanchor='left',
            yanchor='middle'
        )
        
    # Set a white background
    fig.update_layout(plot_bgcolor='white')

    # Modify the size of the figure
    fig.update_layout(width=1500, height=1500)

    # Save the figure as an HTML file
    fig.write_html(f'{output_folder}Wikidata_pathway_barplot_normalized.html')

    # Show or save the figure
    fig.show()

Generate une plot by GENUS and the chemical class

In [34]:
# Define base colors for each Pathway

pathway_colors = {
    'Terpenoids': ('#618264', '#D0E7D2'),  # Green start and lighter green end
    'Alkaloids': ('#305F72', '#5CBCE2'),   # Blue start and lighter blue end  
    'Shikimates and Phenylpropanoids': ('#80558C', '#CBA0AE'),  # Purple start and lighter purple end
    'Polyketides': ('#EF4B4B', '#EC8F6A'),  # Red start and lighter purple end
    'Fatty acids': ('#FF6C22', '#FF9209'),  # Orange start and lighter purple end
    'Amino acids and Peptides': ('#F4E869', '#FAF2D3'),  # Yellow start and lighter purple end
    'Carbohydrates': ('#65451F','#C8AE7D')  # Brown start and lighter purple end
}

def interpolate_color(color1, color2, factor: float):
    """Interpolate between two colors"""
    color1 = np.array(mcolors.to_rgb(color1))
    color2 = np.array(mcolors.to_rgb(color2))
    return mcolors.to_hex((1 - factor) * color1 + factor * color2)

def generate_shades(pathway, num_shades):
    base_color, end_color = pathway_colors.get(pathway, ('gray', 'lightgray'))
    if num_shades == 1:
        return [base_color]  # Return the base color if only one shade is requested
    shades = []
    for i in range(num_shades):
        factor = i / (num_shades - 1)
        shades.append(interpolate_color(base_color, end_color, factor))
    return shades


# Step 1: Initialize an empty DataFrame to store all data
all_data = pd.DataFrame(columns=['organism_taxonomy_08genus', 'chemical_superclass'])

# Step 2: Read data from all .tsv files in the output_folder
genus_data_folder = os.path.join(output_folder, 'genus_data')
for filename in os.listdir(genus_data_folder):
    if filename.endswith(".tsv"):
        file_data = pd.read_csv(os.path.join(genus_data_folder, filename), sep='\t')
        all_data = pd.concat([all_data, file_data])

# Step 3: Rename the "queried_taxaLabel" column to "species"
all_data.rename(columns={'organism_taxonomy_08genus': 'genus'}, inplace=True)

# Remove 'API Error-API Error' and 'Not Classified-Not Classified'
all_data = all_data[(all_data['chemical_superclass'] != 'API Error-API Error') & 
                    (all_data['chemical_superclass'] != 'Not Classified-Not Classified')]

def split_chemical_superclass(row):
    # Check if the value is a string before splitting
    if isinstance(row['chemical_superclass'], str):
        parts = row['chemical_superclass'].split('-')
        if len(parts) == 2:
            return parts[0], parts[1]  # Pathway and Superclass are present
        else:
            return parts[0], 'Unknown'  # Only Pathway is present, or the format is not as expected
    else:
        # Return default values if the entry is NaN or not a string
        return 'Unknown', 'Unknown'

# Apply the function to each row
all_data[['Pathway', 'superclass']] = all_data.apply(lambda row: split_chemical_superclass(row), axis=1, result_type='expand')

# Step 4: Process data for color mapping
color_map = {}
for pathway, superclasses in all_data.groupby('Pathway')['superclass'].unique().items():
    shades = generate_shades(pathway, len(superclasses))
    for superclass, shade in zip(superclasses, shades):
        color_map[f"{pathway}-{superclass}"] = shade

# Step 5: Group and aggregate data to calculate recurrence
agg_data = all_data.groupby(['genus', 'chemical_superclass']).size().reset_index(name='recurrence')

# Step 6: Create the stacked barplot with the custom color palette
fig = px.bar(agg_data, y=agg_data['genus'].apply(lambda x: f"<i>{x}</i>"), x='recurrence',
             title='Stacked Barplot of Predicted Superclasses Ocurrence for Genus',
             labels={'recurrence': 'Recurrence'},
             color='chemical_superclass',
             color_discrete_map=color_map,
             category_orders={'chemical_superclass': sorted(agg_data['chemical_superclass'].unique())},
             orientation='h')

# Modify the y-axis label
fig.update_yaxes(title_text='<i>Genus<i>')

# Set a white background
fig.update_layout(plot_bgcolor='white')

# Modify the size of the figure
fig.update_layout(width=1500, height=1600)

# Save the figure as an HTML file
fig.write_html(f'{output_folder}Wikidata_superclass_barplot_genus.html')

# Show the figure
fig.show()


C:\Users\quirosgu\AppData\Local\Temp\ipykernel_11824\1023424903.py:38: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

C:\Users\quirosgu\AppData\Local\Temp\ipykernel_11824\1023424903.py:38: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

C:\Users\quirosgu\AppData\Local\Temp\ipykernel_11824\1023424903.py:38: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the r

Generate une plot by Genus and the Pathway

In [27]:
# Define custom colors for the 7 pathway categories
pathway_colors = {
    'Terpenoids': '#618264',  # Green start and lighter green end
    'Alkaloids': '#305F72',   # Blue start and lighter blue end
    'Shikimates and Phenylpropanoids': '#80558C',  # Purple start and lighter purple end
    'Polyketides': '#EF4B4B',  # Red start and lighter purple end
    'Fatty acids': '#FF6C22',  # Orange start and lighter purple end
    'Amino acids and Peptides': '#F4E869',  # Yellow start and lighter purple end
    'Carbohydrates': '#65451F' # Brown start and lighter purple end
}

# Step 1: Initialize an empty DataFrame to store all data
all_data = pd.DataFrame(columns=['organism_taxonomy_08genus', 'chemical_superclass'])

# Step 2: Read data from all .tsv files in the output_folder
genus_data_folder = os.path.join(output_folder, 'genus_data')
for filename in os.listdir(genus_data_folder):
    if filename.endswith(".tsv"):
        file_data = pd.read_csv(os.path.join(genus_data_folder, filename), sep='\t')
        all_data = pd.concat([all_data, file_data])

# Step 3: Rename the "queried_taxaLabel" column to "species"
all_data.rename(columns={'organism_taxonomy_08genus': 'genus'}, inplace=True)
all_data.rename(columns={'structure_taxonomy_npclassifier_01pathway': 'Pathway'}, inplace=True)

# Remove 'API Error-API Error' and 'Not Classified-Not Classified'
all_data = all_data[(all_data['Pathway'] != 'API Error') & (all_data['Pathway'] != 'Not Classified')]

# Step 4: Group and aggregate data to calculate recurrence
agg_data = all_data.groupby(['genus', 'Pathway']).size().reset_index(name='recurrence')

# Sort the DataFrame by 'genus' alphabetically
agg_data = agg_data.sort_values(by='genus')

# Step 5: Create the stacked barplot with custom colors
fig = px.bar(agg_data, y=agg_data['genus'].apply(lambda x: f"<i>{x}</i>"), x='recurrence',
             title='Stacked Barplot of Predicted Pathways Ocurrence for Genus',
             labels={'recurrence': 'Recurrence'},
             color='Pathway',
             color_discrete_map=pathway_colors,  # Use custom colors
             category_orders={'genus': sorted(agg_data['genus'].unique())},  # Sort 'genus' alphabetically,
             orientation='h'
            )

# Modify the y-axis label
fig.update_yaxes(title_text='Genus')

# Set a white background
fig.update_layout(plot_bgcolor='white')

# Modify the size of the figure
fig.update_layout(width=1200, height=1400)

# Save the figure as an HTML file
fig.write_html(f'{output_folder}Wikidata_pathway_barplot_genus.html')

# Show or save the figure
fig.show()


/tmp/ipykernel_1352/4133780139.py:20: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/tmp/ipykernel_1352/4133780139.py:20: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/tmp/ipykernel_1352/4133780139.py:20: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.